In [ ]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd

# Otros objetivos
# -----------------------------------------------------------------------
import math

# Gráficos
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
import plotly_express as px


# Métodos estadísticos
# -----------------------------------------------------------------------
from scipy.stats import zscore # para calcular el z-score
from sklearn.neighbors import LocalOutlierFactor # para detectar outliers usando el método LOF
from sklearn.ensemble import IsolationForest # para detectar outliers usando el metodo IF
from sklearn.neighbors import NearestNeighbors # para calcular la epsilon
from sklearn.impute import KNNImputer
# Para generar combinaciones de listas
# -----------------------------------------------------------------------
from itertools import product , combinations
from sklearn.experimental import enable_iterative_imputer  # Necesario para habilitar IterativeImputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

# Gestionar warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append("../../")
from src.SupportOutliers import (
    plot_outliers_univariados,
    identificar_outliers_iqr,
    identificar_outliers_z,
    visualizar_outliers_bivariados,
    explorar_outliers_if,
    explorar_outliers_lof
)

In [ ]:
df = pd.read_csv("../../datos/05_rent_featuredscaled.csv")

# Identificación de outliers

In [ ]:
cols_visualizar = ["status","floor","municipality","hasLift","hasPlan","has3DTour","distancia_centro"]
plot_outliers_univariados(df[cols_visualizar],"b",20,(15,15))

# Veamoslo en un histplot

In [ ]:
cols_visualizar = ["status","floor","municipality","hasLift","hasPlan","has3DTour","distancia_centro"]
plot_outliers_univariados(df[cols_visualizar],"h",20,(15,15))

# Identificar Outliers Multivariados

In [ ]:
cols_visualizar = ["status","floor","municipality","hasLift","hasPlan","has3DTour","distancia_centro","price"]
visualizar_outliers_bivariados(df[cols_visualizar],"price",(20,20))

# Analicemos con IQR si hay outliers
- Solo revisamos las no binarias

In [ ]:
cols_visualizar = ["status","floor","municipality","hasLift","hasPlan","has3DTour","distancia_centro","price"]
dictio_iqr = identificar_outliers_iqr(df[cols_visualizar])

# Realizamos un LOF para detectar outliers

In [ ]:
cols_analizar = ["status","floor","municipality","hasLift","hasPlan","has3DTour","distancia_centro","price"]
df_lof = explorar_outliers_lof(df[cols_visualizar],df,"price",vecinos=[25,60,80],grafica_size=(15,15))

In [ ]:
df_lof.sample()

# Veamos con Isolation Forest

In [ ]:
cols_analizar = ["status","floor","municipality","hasLift","hasPlan","has3DTour","distancia_centro","price"]
df_iso = explorar_outliers_if(df[cols_visualizar],df,"price",grafica_size=(15,15))

In [ ]:
df_iso.sample()

# Categorizamos los outliers de ambas pruebas
- Para comparar resultados
### Vamos a gestionar los outliers, los vamos a clasificar como
- "No es Outlier": Todas las columnas del LOF son 1
- "Improbable Outlier": El 60% de las columnas son 1
- "Es probable que sea Outlier": El 60% de las columnas son -1
- "Outlier Total": El 100% de las columnas son -1

### ¿Qué voy a hacer?
- Voy a contar cuantos 1 tengo en todas las columnas de outliers para poder categorizarla más adelante

#### Saber cuantas columnas de outlier tengo

In [ ]:
len(df_lof.filter(like="outlier_").columns)


In [ ]:
len(df_iso.filter(like="outlier_").columns)

- Tenemos 63 en df_lof
- Tenemos 24 en iso 

### LOF

In [ ]:
# Generamos nueva columna
df_lof["tipo_outliers"] = 0

# Contamos cuantos -1 hay
for columna in df_lof.filter(like="outlier").columns:
    df_lof["tipo_outliers"] += (df_lof[columna] == -1).astype(int)

# Generamos el diccionario de condiciones y textos para reemplazar
condiciones = [
    df_lof["tipo_outliers"] == 0,
    df_lof["tipo_outliers"].between(1, 37,"both"),
    df_lof["tipo_outliers"].between(38, 62,"both"),
    df_lof["tipo_outliers"] == 63
]

textos = [
    "No es Outlier",
    "Improbable Outlier",
    "Probable Outlier",
    "Total Outlier"
]

# Reemplazamos la cuenta por el texto
df_lof["tipo_outliers"] = np.select(condiciones, textos, default="Unknown")

In [ ]:
df_lof.sample()

### Isolation Forest

In [ ]:
# Generamos nueva columna
df_iso["tipo_outliers"] = 0

# Contamos cuantos -1 hay
for columna in df_iso.filter(like="outlier").columns:
    df_iso["tipo_outliers"] += (df_iso[columna] == -1).astype(int)

# Generamos el diccionario de condiciones y textos para reemplazar
condiciones = [
    df_iso["tipo_outliers"] == 0,
    df_iso["tipo_outliers"].between(1, 14,"both"),
    df_iso["tipo_outliers"].between(15, 23,"both"),
    df_iso["tipo_outliers"] == 24
]

textos = [
    "No es Outlier",
    "Improbable Outlier",
    "Probable Outlier",
    "Total Outlier"
]

# Reemplazamos la cuenta por el texto
df_iso["tipo_outliers"] = np.select(condiciones, textos, default="Unknown")

In [ ]:
df_iso.sample()

# Eliminar columnas que ya no necesitamos
- Nos quitamos las columnas de los outliers, ya las hemos categorizado

In [ ]:
df_lof.drop(columns=df_lof.filter(like="outlier_").columns,inplace=True)
df_iso.drop(columns=df_iso.filter(like="outlier_").columns,inplace=True)

# Comparamos Métodos
- Veremos que detecciones han hecho cada uno

In [ ]:
df_lof["tipo_outliers"].value_counts().reset_index()

In [ ]:
df_iso["tipo_outliers"].value_counts().reset_index()

# Utilizaremos el resultado de LOF
- Vamos a volver nulo el valor de todo lo que tenga como mínimo un resultado de que es un outlier

In [ ]:
df_lof = df_lof[df_lof["tipo_outliers"] != "Total Outlier"]
df_lof["tipo_outliers"].unique()
df_lof.reset_index(drop=True,inplace=True)   

In [ ]:
cols_nulificar = ["status","floor","municipality","hasLift","hasPlan","has3DTour","distancia_centro"]
for columna in cols_nulificar:
    df_lof.loc[df_lof["tipo_outliers"] == "Improbable Outlier", columna] = np.nan
    df_lof.loc[df_lof["tipo_outliers"] == "Probable Outlier", columna] = np.nan
    print(f"Se han vuelto nulos: {df_lof[columna].isnull().sum()} valores en {columna}")



# Rellenamos los nulos con Random Forest en Iterative

In [ ]:
df = df_lof.copy()

In [ ]:
df.isnull().sum()

In [ ]:
cols_nulificar = ["status","floor","municipality","hasLift","hasPlan","has3DTour","distancia_centro","price"]
df_num = df[cols_nulificar]

# Random Forest
iterative_imputer = IterativeImputer(
    estimator=RandomForestRegressor(random_state=42),
    #max_iter=10,  # Número de iteraciones para ajustar los valores
    random_state=42
)

df_num_imputado = iterative_imputer.fit_transform(df_num)
df_num_sin_nulos = pd.DataFrame(df_num_imputado, columns=df_num.columns)
df_rf = df.copy()
df_rf[df_num.columns] = df_num_sin_nulos
df_rf[cols_nulificar].isnull().sum()


# Eliminamos "tipo_outliers"
- Ya la hemos utilizado como queríamos

In [ ]:
df_rf.drop(columns=("tipo_outliers"),inplace=True)

# Guardamos 
- Nos vamos a intentar una regresión Lineal

In [ ]:
df_rf.to_csv("../../datos/06_rent_outliers_gestionados.csv",index=False)